In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Load Dataset
def load_data():
    # Simulated dataset
    data = {
        'tenure': np.random.randint(1, 72, 1000),
        'MonthlyCharges': np.random.uniform(20, 100, 1000),
        'TotalCharges': np.random.uniform(20, 7200, 1000),
        'Contract': np.random.choice(['Month-to-Month', 'One year', 'Two year'], 1000),
        'Churn': np.random.choice([0, 1], 1000)
    }
    df = pd.DataFrame(data)
    df = pd.get_dummies(df, columns=['Contract'], drop_first=True)
    return df

# Data Preprocessing
def preprocess_data(df):
    X = df.drop('Churn', axis=1)
    y = df['Churn']
    scaler = MinMaxScaler()
    X[['tenure', 'MonthlyCharges', 'TotalCharges']] = scaler.fit_transform(
        X[['tenure', 'MonthlyCharges', 'TotalCharges']]
    )
    return train_test_split(X, y, test_size=0.2, random_state=42), scaler

# Build Model
def build_model(input_dim):
    model = Sequential([
        Dense(64, input_dim=input_dim, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Model
def train_model(model, X_train, y_train, X_val, y_val):
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, verbose=1)
    return model

# Evaluate Model
def evaluate_model(model, X_test, y_test):
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    print("\nEvaluation Metrics:")
    print(classification_report(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))

# Real-Time Prediction with Limited Inputs
def predict_churn_simple(model, scaler):
    print("Enter customer details for prediction:")
    
    # Collect inputs
    tenure = float(input("Enter Tenure (in months): "))
    monthly_charges = float(input("Enter Monthly Charges ($): "))
    total_charges = tenure * monthly_charges  # Approximation if not provided
    contract_type = input("Enter Contract Type (Month-to-Month/One year/Two year): ")
    
    # Convert contract type to dummy variables
    contract_features = {
        'Month-to-Month': [1, 0],
        'One year': [0, 1],
        'Two year': [0, 0]
    }
    contract = contract_features.get(contract_type, [0, 0])  # Default to 'Two year' if invalid input
    
    # Prepare input data
    input_data = pd.DataFrame([{
        'tenure': tenure,
        'MonthlyCharges': monthly_charges,
        'TotalCharges': total_charges,
        'Contract_One year': contract[0],
        'Contract_Two year': contract[1]
    }])
    
    # Scale numeric features
    input_data[['tenure', 'MonthlyCharges', 'TotalCharges']] = scaler.transform(
        input_data[['tenure', 'MonthlyCharges', 'TotalCharges']]
    )
    
    # Predict churn
    prediction_prob = model.predict(input_data).flatten()[0]
    prediction = "Yes" if prediction_prob > 0.5 else "No"
    print(f"\nPrediction: {prediction}")
    print(f"Churn Probability: {prediction_prob:.2f}")
    
    # Provide retention strategies if churn predicted
    if prediction == "Yes":
        print("\nRetention Strategies:")
        if tenure < 12:
            print("- Provide loyalty incentives (e.g., discounts, free upgrades) to increase tenure.")
        if monthly_charges > 70:
            print("- Offer a discount or bundle services to reduce monthly costs.")
        if contract_type == 'Month-to-Month':
            print("- Encourage switching to a longer-term contract with exclusive benefits.")

# Main Function
def main():
    df = load_data()
    (X_train, X_test, y_train, y_test), scaler = preprocess_data(df)
    
    # Build and train model
    model = build_model(X_train.shape[1])
    model = train_model(model, X_train, y_train, X_test, y_test)
    
    # Evaluate model
    evaluate_model(model, X_test, y_test)
    
    # Real-Time Prediction
    predict_churn_simple(model, scaler)

# Run the Program
if __name__ == '__main__':
    main()
